### 12. Different Size Transformation
The process of image registration can be made faster, when smaller version of the fixed and moving images are used for calculation of the transformation parameters. These parameters can subsequently be used for the transformation of larger images. 

In this example image generators are used to get small and larger versions of hypothetical fixed and moving images. An important note is that the width and the height of an image should remain the same. Downsampling of an image means decreasing the number of pixels by increasing the size (or spacing) of a pixel and thereby remaining the width and height of the image the same.

### Elastix

In [1]:
# First two import are currently necessary to run ITKElastix on MacOs
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# To generate a downsampled smaller image, the spacing of the image should be increased 10-fold in both directions,
# when the number of pixels is decreased 10-fold in both directions.
def image_generator(x1, x2, y1, y2, downsampled=False):
    if downsampled:
        image = np.zeros([100, 100], np.float32)
    else:
        image = np.zeros([1000, 1000], np.float32)
    image[y1:y2, x1:x2] = 1
    image_ikt = itk.image_view_from_array(image)
    if downsampled:
        image_ikt.SetSpacing([10,10])
    return image_ikt

In [3]:
# Create small images for registration
fixed_image_small = image_generator(25,75,25,75, downsampled=True)
moving_image_small = image_generator(0,45,10,75, downsampled=True)

# .. and a big moving image for transformation
moving_image_large = image_generator(0,450,100,750)

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('affine')
parameter_object.AddParameterMap(parameter_map_rigid)

Registration with the registration function...

In [4]:
# Call registration function
result_image_small, result_transform_parameters = itk.elastix_registration_method(
    fixed_image_small, moving_image_small,
    parameter_object=parameter_object)

### Transformix

In order for the registration to work, the size of the image in the transform parameter file has to be adjusted to the correct size and spacing of the large image.

In [5]:
result_transform_parameters.SetParameter(0, "Size", ['1000', '1000'])
result_transform_parameters.SetParameter(0, "Spacing", ['1', '1'])

Transformation can either be done in one line with the transformix function...

In [6]:
result_image_large = itk.transformix_filter(
    moving_image=moving_image_large,
    transform_parameter_object=result_transform_parameters,
    log_to_console=False)

.. or by initiating an transformix image filter object.

In [7]:
# Load Transformix Object
transformix_object = itk.TransformixFilter.New(moving_image_large)
transformix_object.SetTransformParameterObject(result_transform_parameters)

# Update object (required)
transformix_object.UpdateLargestPossibleRegion()

# Results of Transformation
result_image_large = transformix_object.GetOutput()